In [2]:
import os
import re
import time
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split

pd.set_option("display.max_rows", None, "display.max_columns", None)

In [49]:
path = os.path.abspath('../../data/teams.csv')
df = pd.read_csv(path)
df.head()

,School,"City, State",SR key,NCAA key,NCAA School,NCAA Name,background-color
0,Abilene Christian,"Abilene, Texas",abilene-christian,abilene-christian,Abilene Christian,Abilene Christian University,#582C83
1,Air Force,"USAF Academy, Colorado",air-force,air-force,Air Force,Air Force Academy,#0032A0
2,Akron,"Akron, Ohio",akron,akron,Akron,University of Akron,#0F192B
3,Alabama,"Tuscaloosa, Alabama",alabama,alabama,Alabama,University of Alabama,#9D2235
4,Alabama A&M,"Normal, Alabama",alabama-am,alabama-am,Alabama A&M,Alabama A&M University,#862633


In [50]:
SEASON = 2023
SR_SCHOOL_KEYS = [row.to_dict().get('SR key') for index, row in df.iterrows()]

get_gamelog_basic_url = lambda school_key : f'https://www.sports-reference.com/cbb/schools/{school_key}/men/{SEASON}-gamelogs.html'
get_gamelog_advanced_url = lambda school_key : f'https://www.sports-reference.com/cbb/schools/{school_key}/men/{SEASON}-gamelogs-advanced.html'

## Download gamelogs HTML
Download the basic and advanced gamelog html for each team

In [51]:
for school in tqdm(SR_SCHOOL_KEYS):
    basic_url, advanced_url = get_gamelog_basic_url(school), get_gamelog_advanced_url(school)

    # time.sleep(3) # Delay for 3 seconds
    # basic_html = requests.get(basic_url).content
    time.sleep(3) # Delay for 3 seconds
    advanced_html = requests.get(advanced_url).content

    basic_file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_basic.html')
    advanced_file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_advanced.html')

    dir = os.path.dirname(basic_file_path)
    if not os.path.exists(dir):
        os.makedirs(dir)

    # with open(basic_file_path, 'w') as file:
    #     file.write(basic_html.decode('utf-8'))
    with open(advanced_file_path, 'w') as file:
        file.write(advanced_html.decode('utf-8'))

100%|██████████| 491/491 [29:41<00:00,  3.63s/it]


## Basic gamelog CSV
Extract the basic gamelog to csv

In [52]:
for school in tqdm(SR_SCHOOL_KEYS):
    try:
        file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_basic.html')

        team_df = pd.read_html(file_path)[0]

        # drop columns from 'Defensive Four Factors'
        opponent_columns = [column for column in team_df.columns if 'Opponent' in column[0]]
        team_df = team_df.drop(opponent_columns, axis=1)

        # Use second level column names
        team_df.columns = [column[1] for column in team_df.columns]

        # rename to location column
        team_df = team_df.rename(columns={'Unnamed: 2_level_1': 'Location'})

        # remove unneeded columns
        unneeded_columns = [column for column in team_df.columns if 'Unnamed' in column] + ['G']
        team_df = team_df.drop(unneeded_columns, axis=1)

        # Drop repeating header rows
        team_df = team_df[team_df.Tm != 'Tm']
        team_df = team_df[team_df.FG != 'School']

        # rename repeating 'Opp' column
        index = team_df.columns.to_list().index('Opp')
        team_df.columns.values[index] = 'Opp name'

        # save file
        csv_file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_basic.csv')
        team_df.to_csv(csv_file_path, index=False)
    except ValueError:
        pass

100%|██████████| 491/491 [00:48<00:00, 10.12it/s]


Quick visual verification:

In [53]:
school = 'connecticut'
file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_basic.csv')
team_df = pd.read_csv(file_path)
team_df.head()

,Date,Location,Opp name,W/L,Tm,Opp,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,2022-11-07,NaN,Stonehill,W,85,54,29,63,0.460,5,24,0.208,22,31,0.710,15,40,16,11,8,15,20
1,2022-11-11,NaN,Boston University,W,86,57,31,55,0.564,10,21,0.476,14,26,0.538,11,40,17,4,3,14,18
2,2022-11-15,NaN,Buffalo,W,84,64,28,63,0.444,6,23,0.261,22,28,0.786,12,37,17,11,6,15,13
3,2022-11-18,NaN,UNC Wilmington,W,86,50,28,52,0.538,10,22,0.455,20,23,0.870,5,33,20,3,5,10,19
4,2022-11-20,NaN,Delaware State,W,95,60,35,61,0.574,12,28,0.429,13,19,0.684,7,34,25,9,6,12,20


## Advanced gamelog CSV
Extract the advanced gamelog to csv

In [54]:
for school in tqdm(SR_SCHOOL_KEYS):
    try:
        file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_advanced.html')

        team_df = pd.read_html(file_path)[0]

        # drop columns from 'Defensive Four Factors'
        defensive_columns = [column for column in team_df.columns if 'Defensive' in column[0]]
        team_df = team_df.drop(defensive_columns, axis=1)

        # Use second level column names
        team_df.columns = [column[1] for column in team_df.columns]

        # rename to location column
        team_df = team_df.rename(columns={'Unnamed: 2_level_1': 'Location'})

        # remove unneeded columns
        unneeded_columns = [column for column in team_df.columns if 'Unnamed' in column] + ['G']
        team_df = team_df.drop(unneeded_columns, axis=1)

        # Drop repeating header rows
        team_df = team_df[team_df.Tm != 'Tm']
        team_df = team_df[team_df['eFG%'] != 'Offensive Four Factors']

        # rename repeating 'Opp' column
        index = team_df.columns.to_list().index('Opp')
        team_df.columns.values[index] = 'Opp name'

        # save file
        csv_file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_advanced.csv')
        team_df.to_csv(csv_file_path, index=False)
    except ValueError:
        pass

100%|██████████| 491/491 [00:44<00:00, 10.98it/s]


Quick visual verification:

In [55]:
school = 'connecticut'
file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_advanced.csv')
team_df = pd.read_csv(file_path)
team_df.head()

,Date,Location,Opp name,W/L,Tm,Opp,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
0,2022-11-07,NaN,Stonehill,W,85,54,111.8,71.1,76.1,0.492,0.381,0.547,66.7,55.2,14.5,28.6,0.500,16.2,48.4,0.349
1,2022-11-11,NaN,Boston University,W,86,57,122.9,81.4,70.4,0.473,0.382,0.638,58.8,54.8,5.7,7.3,0.655,17.2,34.4,0.255
2,2022-11-15,NaN,Buffalo,W,84,64,106.3,81.0,78.7,0.444,0.365,0.550,58.7,60.7,13.9,14.6,0.492,16.4,35.3,0.349
3,2022-11-18,NaN,UNC Wilmington,W,86,50,128.4,74.6,66.7,0.442,0.423,0.683,55.0,71.4,4.5,12.5,0.635,13.7,23.8,0.385
4,2022-11-20,NaN,Delaware State,W,95,60,125.0,78.9,75.7,0.311,0.459,0.678,57.6,71.4,11.8,14.0,0.672,14.6,26.9,0.213


## Change Opp name column to Opp key
The Opp name does not always match the school name we have saved in the team data, thus we will reparse the tables to instead use the SR keys

In [56]:
for school in tqdm(SR_SCHOOL_KEYS):
    try:
        school_keys = []
        html_file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_basic.html')

        with open(html_file_path, 'r') as file:
            soup = BeautifulSoup(file, 'html.parser')
            table = soup.find("table")
            rows = table.find_all('tr')
        
            for row in rows[2:]:
                try:
                    link = row.find_all('td')[2].find('a')['href']
                    key = re.search(r'/schools/([^/]+)/', link).group(1)
                    school_keys.append(key)
                except IndexError:
                    # repeating header row
                    pass

        for dataset in ['basic', 'advanced']:
            csv_file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_{dataset}.csv')
            team_df = pd.read_csv(csv_file_path)

            # Shape for both must match same rows
            assert team_df.shape[0] == len(school_keys)

            team_df['Opp name'] = school_keys
            team_df = team_df.rename(columns={'Opp name': 'Opp key'})

            team_df.to_csv(csv_file_path, index=False)
    except AttributeError:
        pass

100%|██████████| 491/491 [00:37<00:00, 13.02it/s]


Quick visual verification:

In [57]:
school = 'connecticut'
file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_basic.csv')
team_df = pd.read_csv(file_path)
team_df.head()

,Date,Location,Opp key,W/L,Tm,Opp,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,2022-11-07,NaN,stonehill,W,85,54,29,63,0.460,5,24,0.208,22,31,0.710,15,40,16,11,8,15,20
1,2022-11-11,NaN,boston-university,W,86,57,31,55,0.564,10,21,0.476,14,26,0.538,11,40,17,4,3,14,18
2,2022-11-15,NaN,buffalo,W,84,64,28,63,0.444,6,23,0.261,22,28,0.786,12,37,17,11,6,15,13
3,2022-11-18,NaN,north-carolina-wilmington,W,86,50,28,52,0.538,10,22,0.455,20,23,0.870,5,33,20,3,5,10,19
4,2022-11-20,NaN,delaware-state,W,95,60,35,61,0.574,12,28,0.429,13,19,0.684,7,34,25,9,6,12,20


In [58]:
school = 'connecticut'
file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_advanced.csv')
team_df = pd.read_csv(file_path)
team_df.head()

,Date,Location,Opp key,W/L,Tm,Opp,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
0,2022-11-07,NaN,stonehill,W,85,54,111.8,71.1,76.1,0.492,0.381,0.547,66.7,55.2,14.5,28.6,0.500,16.2,48.4,0.349
1,2022-11-11,NaN,boston-university,W,86,57,122.9,81.4,70.4,0.473,0.382,0.638,58.8,54.8,5.7,7.3,0.655,17.2,34.4,0.255
2,2022-11-15,NaN,buffalo,W,84,64,106.3,81.0,78.7,0.444,0.365,0.550,58.7,60.7,13.9,14.6,0.492,16.4,35.3,0.349
3,2022-11-18,NaN,north-carolina-wilmington,W,86,50,128.4,74.6,66.7,0.442,0.423,0.683,55.0,71.4,4.5,12.5,0.635,13.7,23.8,0.385
4,2022-11-20,NaN,delaware-state,W,95,60,125.0,78.9,75.7,0.311,0.459,0.678,57.6,71.4,11.8,14.0,0.672,14.6,26.9,0.213


## Combine basic and advanced gamelog CSVs

In [59]:
for school in tqdm(SR_SCHOOL_KEYS):
    try:
        basic_file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_basic.csv')
        advanced_file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_advanced.csv')

        basic_team_df, advanced_team_df = pd.read_csv(basic_file_path), pd.read_csv(advanced_file_path)

        merged_team_df = pd.merge(basic_team_df, advanced_team_df, on=['Date', 'Location', 'Opp key', 'W/L', 'Tm', 'Opp'])

        # fill NaN location values to 'H' to represent Home
        merged_team_df['Location'] = merged_team_df['Location'].fillna('H')
        
        csv_file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_merged.csv')
        merged_team_df.to_csv(csv_file_path, index=False)
    except FileNotFoundError:
        pass

100%|██████████| 491/491 [00:05<00:00, 89.53it/s] 


Quick visual verification:

In [60]:
school = 'connecticut'
file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_merged.csv')
team_df = pd.read_csv(file_path)
team_df.head()

,Date,Location,Opp key,W/L,Tm,Opp,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA
0,2022-11-07,H,stonehill,W,85,54,29,63,0.460,5,24,0.208,22,31,0.710,15,40,16,11,8,15,20,111.8,71.1,76.1,0.492,0.381,0.547,66.7,55.2,14.5,28.6,0.500,16.2,48.4,0.349
1,2022-11-11,H,boston-university,W,86,57,31,55,0.564,10,21,0.476,14,26,0.538,11,40,17,4,3,14,18,122.9,81.4,70.4,0.473,0.382,0.638,58.8,54.8,5.7,7.3,0.655,17.2,34.4,0.255
2,2022-11-15,H,buffalo,W,84,64,28,63,0.444,6,23,0.261,22,28,0.786,12,37,17,11,6,15,13,106.3,81.0,78.7,0.444,0.365,0.550,58.7,60.7,13.9,14.6,0.492,16.4,35.3,0.349
3,2022-11-18,H,north-carolina-wilmington,W,86,50,28,52,0.538,10,22,0.455,20,23,0.870,5,33,20,3,5,10,19,128.4,74.6,66.7,0.442,0.423,0.683,55.0,71.4,4.5,12.5,0.635,13.7,23.8,0.385
4,2022-11-20,H,delaware-state,W,95,60,35,61,0.574,12,28,0.429,13,19,0.684,7,34,25,9,6,12,20,125.0,78.9,75.7,0.311,0.459,0.678,57.6,71.4,11.8,14.0,0.672,14.6,26.9,0.213


## Generating Moving Averages
Next we will generate a csv adding the moving averages for each statistic

In [61]:
SPAN = 5

META_LABELS = ['Date', 'Location', 'Opp key', 'W/L', 'Tm', 'Opp']
STAT_LABELS = ['FG', 'FGA', 'FG%','3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA']

In [62]:
for school in tqdm(SR_SCHOOL_KEYS):
    try:
        file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_merged.csv')
        team_df = pd.read_csv(file_path)

        for column in team_df.columns:
            if column in META_LABELS:
                continue

            # Simple moving averages
            team_df[f"{column}_SMA"] = team_df.loc[:, column].rolling(window=SPAN).mean()
            team_df[f"{column}_SMA"] = team_df[f"{column}_SMA"].shift(1)

            # Cumulative moving average
            team_df[f"{column}_CMA"] = team_df.loc[:, column].expanding(min_periods=SPAN).mean()
            team_df[f"{column}_CMA"] = team_df[f"{column}_CMA"].shift(1)

            # Exponential moving average
            team_df[f"{column}_EMA"] = team_df.loc[:, column].ewm(span=SPAN, adjust=False).mean()
            team_df[f"{column}_EMA"] = team_df[f"{column}_EMA"].shift(1)
        
        # Drop any rows with NULL values
        team_df.dropna(inplace=True)
        
        ma_file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_ma.csv')
        team_df.to_csv(ma_file_path, index=False)
    except FileNotFoundError:
        continue

100%|██████████| 491/491 [00:17<00:00, 28.37it/s]


Quick visual verification:

In [63]:
school = 'connecticut'
file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_ma.csv')
team_df = pd.read_csv(file_path)
team_df.head()

,Date,Location,Opp key,W/L,Tm,Opp,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,ORtg,DRtg,Pace,FTr,3PAr,TS%,TRB%,AST%,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,FG_SMA,FG_CMA,FG_EMA,FGA_SMA,FGA_CMA,FGA_EMA,FG%_SMA,FG%_CMA,FG%_EMA,3P_SMA,3P_CMA,3P_EMA,3PA_SMA,3PA_CMA,3PA_EMA,3P%_SMA,3P%_CMA,3P%_EMA,FT_SMA,FT_CMA,FT_EMA,FTA_SMA,FTA_CMA,FTA_EMA,FT%_SMA,FT%_CMA,FT%_EMA,ORB_SMA,ORB_CMA,ORB_EMA,TRB_SMA,TRB_CMA,TRB_EMA,AST_SMA,AST_CMA,AST_EMA,STL_SMA,STL_CMA,STL_EMA,BLK_SMA,BLK_CMA,BLK_EMA,TOV_SMA,TOV_CMA,TOV_EMA,PF_SMA,PF_CMA,PF_EMA,ORtg_SMA,ORtg_CMA,ORtg_EMA,DRtg_SMA,DRtg_CMA,DRtg_EMA,Pace_SMA,Pace_CMA,Pace_EMA,FTr_SMA,FTr_CMA,FTr_EMA,3PAr_SMA,3PAr_CMA,3PAr_EMA,TS%_SMA,TS%_CMA,TS%_EMA,TRB%_SMA,TRB%_CMA,TRB%_EMA,AST%_SMA,AST%_CMA,AST%_EMA,STL%_SMA,STL%_CMA,STL%_EMA,BLK%_SMA,BLK%_CMA,BLK%_EMA,eFG%_SMA,eFG%_CMA,eFG%_EMA,TOV%_SMA,TOV%_CMA,TOV%_EMA,ORB%_SMA,ORB%_CMA,ORB%_EMA,FT/FGA_SMA,FT/FGA_CMA,FT/FGA_EMA
0,2022-11-24,N,oregon,W,83,59,30,63,0.476,17,37,0.459,6,11,0.545,11,35,22,9,7,11,25,118.6,84.3,70.1,0.175,0.587,0.608,57.4,73.3,12.9,21.9,0.611,13.9,32.4,0.095,30.2,30.200000,30.827160,58.8,58.800000,59.098765,0.5160,0.516000,0.523235,8.6,8.600000,9.086420,23.6,23.600000,24.444444,0.3658,0.365800,0.370877,18.2,18.200000,17.765432,25.4,25.400000,24.283951,0.7176,0.717600,0.731160,10.0,10.000000,9.271605,36.8,36.800000,36.000000,19.0,19.000000,20.135802,7.6,7.600000,7.864198,5.6,5.600000,5.876543,13.2,13.200000,12.790123,18.0,18.000000,18.543210,118.88,118.880000,120.170370,77.40,77.400000,76.961728,73.52,73.520000,73.700000,0.4324,0.432400,0.411568,0.4020,0.402000,0.414062,0.6192,0.619200,0.630321,59.36,59.360000,59.101235,62.70,62.700000,64.975309,10.08,10.080000,10.419753,15.40,15.400000,15.977778,0.5908,0.590800,0.601457,15.62,15.620000,15.239506,33.76,33.760000,32.443210,0.3102,0.310200,0.302383
1,2022-11-25,N,alabama,W,82,67,26,60,0.433,9,24,0.375,21,24,0.875,8,30,18,8,7,11,23,110.8,90.5,74.1,0.400,0.400,0.574,48.4,69.2,10.8,20.6,0.508,13.3,24.2,0.350,30.4,30.166667,30.551440,58.8,59.500000,60.399177,0.5192,0.509333,0.507490,11.0,10.000000,11.724280,26.2,25.833333,28.629630,0.4160,0.381333,0.400251,15.0,16.166667,13.843621,21.4,23.000000,19.855967,0.6846,0.688833,0.669107,9.2,10.166667,9.847737,35.8,36.500000,35.666667,20.2,19.500000,20.757202,7.2,7.833333,8.242798,5.4,5.833333,6.251029,12.4,12.833333,12.193416,19.0,19.166667,20.695473,120.24,118.833333,119.646914,80.04,78.550000,79.407819,72.32,72.950000,72.500000,0.3690,0.389500,0.332712,0.4432,0.432833,0.471708,0.6314,0.617333,0.622881,57.50,59.033333,58.534156,66.32,64.466667,67.750206,9.76,10.550000,11.246502,14.06,16.483333,17.951852,0.6130,0.594167,0.604638,15.16,15.333333,14.793004,30.56,33.533333,32.428807,0.2594,0.274333,0.233255
2,2022-11-27,N,iowa-state,W,71,53,22,53,0.415,7,26,0.269,20,25,0.800,18,45,16,5,2,17,15,112.7,84.1,62.8,0.472,0.491,0.547,70.3,72.7,7.9,4.9,0.481,20.8,60.0,0.377,29.4,29.571429,29.034294,59.8,59.571429,60.266118,0.4930,0.498429,0.482660,10.8,9.857143,10.816187,26.8,25.571429,27.086420,0.3958,0.380429,0.391834,16.4,16.857143,16.229081,21.0,23.142857,21.237311,0.7520,0.715429,0.737738,8.6,9.857143,9.231824,33.8,35.571429,33.777778,20.4,19.285714,19.838134,8.0,7.857143,8.161866,6.2,6.000000,6.500686,11.8,12.571429,11.795610,20.0,19.714286,21.463649,117.82,117.685714,116.697942,81.86,80.257143,83.105213,73.06,73.114286,73.033333,0.3544,0.391000,0.355141,0.4468,0.428143,0.447805,0.6186,0.611143,0.606587,55.42,57.514286,55.156104,69.20,65.142857,68.233471,10.78,10.585714,11.097668,16.72,17.071429,18.834568,0.5836,0.581857,0.572425,14.38,15.042857,14.295336,28.52,32.200000,29.685871,0.2784,0.285143,0.272170
3,2022-12-01,H,oklahoma-state,W,74,64,21,56,0.375,8,23,0.348,24,33,0.727,10,28,11,8,3,6,16,107.2,92.8,68.9,0.589,0.411,0.516,45.2,52.4,11.6,6.7,0.446,7.7,30.3,0.429,28.2,28.625000,26.689529,57.8,58.750000,57.844079,0.4872,0.488000,0.460107,11.0,9.500000,9.544124,27.4,25.625000,26.724280,0.3974,0.366500,0.350889,16.0,17.250000,17.486054,20.4,23.3

## Merge opponent data

In [64]:
all_stat_cols = [item for col in STAT_LABELS for item in [col, f'{col}_SMA', f'{col}_CMA', f'{col}_EMA']]
rename_opposing_cols = {item: f'opp_{item}' for stat in STAT_LABELS for item in [f'{stat}', f'{stat}_SMA', f'{stat}_CMA', f'{stat}_EMA']}

for school in tqdm(SR_SCHOOL_KEYS):
    try:
        file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_ma.csv')
        team_df = pd.read_csv(file_path)

        if team_df.shape[0] > 0:
            home_df, away_df = pd.DataFrame(), pd.DataFrame()
            for index, row in team_df.iterrows():
                try:
                    game_obj = row.to_dict()
                    opponent_key = game_obj.get('Opp key')

                    opponent_file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{opponent_key}/{opponent_key}_ma.csv')
                    opponent_df = pd.read_csv(opponent_file_path)

                    opponent_df = opponent_df.loc[(opponent_df['Opp key'] == school) & (opponent_df['Date'] == game_obj.get('Date'))]
                    current_df = team_df[(team_df['Opp key'] == game_obj.get('Opp key')) & (team_df['Date'] == game_obj.get('Date'))]

                    if game_obj.get('Location') == '@':
                        home_df, away_df = pd.concat([home_df, opponent_df]), pd.concat([away_df, current_df])
                    else:
                        home_df, away_df = pd.concat([home_df, current_df]), pd.concat([away_df, opponent_df])
                except FileNotFoundError:
                    continue

            # flip score column names for away dataframe to match home dataframe
            away_df.rename(columns={'Tm': 'Opp', 'Opp': 'Tm'}, inplace=True)

            away_df = away_df.drop(['Location', 'Opp key', 'W/L'], axis=1)
            away_df.rename(columns=rename_opposing_cols, inplace=True)

            merged_df = pd.merge(home_df, away_df, on=["Date", "Tm", "Opp"])
            merged_df = merged_df.sort_values(by='Date')

            merged_file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_full.csv')
            merged_df.to_csv(merged_file_path, index=False)
        else:
            continue
    except FileNotFoundError:
        continue

100%|██████████| 491/491 [00:49<00:00, 10.01it/s]


## Generate training data

In [65]:
all_data_df = pd.DataFrame()

for school in tqdm(SR_SCHOOL_KEYS):
    file_path = os.path.abspath(f'../../data/seasons/{SEASON}/{school}/{school}_full.csv')

    if os.path.exists(file_path):
        team_df = pd.read_csv(file_path)
        all_data_df = pd.concat([all_data_df, team_df])
    else:
        continue

# sort by date
all_data_df = all_data_df.sort_values(by="Date")

# Add feature for if game is neutral site
all_data_df['Neutral'] = all_data_df['Location'].apply(lambda x: 1 if x == 'N' else 0)

# add label for win (1 = win, 0 = loss)
all_data_df['Win'] = (all_data_df['Tm'] > all_data_df['Opp']).astype(int)

# remove meta columns
all_data_df = all_data_df.drop(META_LABELS, axis=1)

# remove non moving average columns
all_data_df = all_data_df.drop(STAT_LABELS, axis=1)
all_data_df = all_data_df.drop([f'opp_{col}' for col in STAT_LABELS], axis=1)

# Drop any rows with NULL value
all_data_df.dropna(inplace=True)

# Drop any duplicate rows
all_data_df.drop_duplicates(inplace=True)

training_data_path = os.path.abspath(f'../../data/{SEASON}_training_data.csv')
all_data_df.to_csv(training_data_path, index=False)

100%|██████████| 491/491 [00:05<00:00, 83.43it/s] 


## Create full train test split
A typical train-test split for machine learning models is to use 70% of the data for training and 30% for testing.

In [6]:
data_dir_path = os.path.abspath(f'../../data/')
pattern = r"\d{4}_training_data\.csv"
filenames = [filename for filename in os.listdir(data_dir_path) if re.match(pattern, filename)]
merged_df = pd.concat([pd.read_csv(os.path.join(data_dir_path, filename)) for filename in filenames], ignore_index=True)

print(f'Original shape: {merged_df.shape}')
merged_df.dropna(inplace=True)
print(f'Shape after dropping rows with null values: {merged_df.shape}')
merged_df.drop_duplicates(inplace=True)
print(f'Shape after dropping duplicate rows: {merged_df.shape}')

train_df, test_df = train_test_split(merged_df, test_size=0.3)

print(len(train_df), 'train examples')
print(len(test_df), 'test examples')

train_df.to_csv(os.path.join(data_dir_path, 'training_set.csv'))
test_df.to_csv(os.path.join(data_dir_path, 'testing_set.csv'))

Original shape: (18354, 182)
Shape after dropping rows with null values: (18354, 182)
Shape after dropping duplicate rows: (18354, 182)
12847 train examples
5507 test examples
